In [ ]:
from mlflow.tracking import MlflowClient
import pandas as pd
mlflow_path = "/home/davina/Private/repos/CRRT/mlruns"
client = MlflowClient(mlflow_path)

In [ ]:
# exclude  // tune trial and //eval best
runs = client.search_runs(
    experiment_ids=client.get_experiment_by_name("static_learning").experiment_id,
    filter_string='tags.mlflow.runName="rolling-window"'
)

In [ ]:
from datetime import datetime, timezone, timedelta
lookback = 1
each_window_results = [
    run for run in runs
    if (run.data.tags.get("slide_window_by", None) is not None)
    and (datetime.now() - datetime.fromtimestamp(run.info.start_time/1000) <= timedelta(days=lookback))
]

In [ ]:
metrics_over_windows = pd.DataFrame(
    [run.data.metrics for run in each_window_results],
    index=[run.data.tags["slide_window_by"] for run in each_window_results]
).rename_axis(index="Slide").sort_index(axis=0)
# metrics_over_windows.columns = pd.MultiIndex.from_product([["Metrics"], metrics_over_windows.columns])
metrics_over_windows

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# g = sns.lineplot(data=metrics_over_windows)
# g = sns.relplot(data=metrics_over_windows, kind="line")
# g.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)
# plt.show()
g = sns.FacetGrid(
    metrics_over_windows.melt(ignore_index=False,var_name="Metrics").reset_index(),
    col="Metrics", col_wrap=4, sharey=False
)
g.map(sns.lineplot, "Slide", "value")

In [ ]:
# cleanup
# for run in runs:
#     client.delete_run(run.info.run_id)